<a href="https://colab.research.google.com/github/tejaswishetty17/Large-Language-Models-Step-by-Step/blob/main/3_Looking_inside_LLms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers>=4.41.2 accelerate>=0.31.0



> Using a gated model from huggingface



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import os

access_token = ""

#Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", token = access_token)


model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
    token = access_token
)

In [24]:
#create a pipeline
generation_gated = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    max_new_tokens=1000
    )

Device set to use cuda




> Lets see the Inputs and Outputs of a Trained Transformer LLM



In [30]:
prompt = [
    {"role":"user", "content":"Write a mail to send my resume to a HR who is recruiting for Data Scientist role. Also, add my github project details."}
  ]
output = generation_gated(prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [39]:
print(output[0]['generated_text'][1]['content'])

 Dear [HR Name],

I am writing to express my interest in the Data Scientist role that you are currently recruiting for. I am excited about the opportunity to join your team and contribute my skills and experience to help drive your business forward.

I have attached my resume, which provides a detailed overview of my education, work experience, and skills. I have a strong background in data analysis, machine learning, and statistical modeling, and I am confident that I have the skills and experience necessary to excel in this role.

In addition to my resume, I would like to share with you some of my recent projects that I have worked on. I have a GitHub account where you can view my work, including a project on predicting customer churn for a telecom company and a project on analyzing customer sentiment for an e-commerce company. You can find my GitHub account at [insert link].

Thank you for considering my application. I look forward to the opportunity to discuss my qualifications fur

In [40]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0



> Lets see how the single token from a probability distribution is choosen(sampling/decoding)



In [110]:
prompt = "The capital of Japan is"

#Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors='pt').input_ids
input_ids = input_ids.to("cuda")

#Get the output of the model before the lm_head
model_output = model.model(input_ids)
print("model_output: \n", model_output)

#Get the output from the lm_head
lm_head_output = model.lm_head(model_output[0])
print("\n\n lm_head_output: \n", lm_head_output)

model_output: 
 BaseModelOutputWithPast(last_hidden_state=tensor([[[-2.0469,  1.9531, -2.1250,  ..., -0.1250, -2.2656,  2.9531],
         [ 0.6953,  3.4531,  1.4688,  ..., -4.3125,  1.0234,  0.4258],
         [-7.0625, -1.6641,  4.0938,  ..., -4.3750, -2.0625, -2.6250],
         [-2.2969,  2.5469, -0.1289,  ..., -2.0156,  2.3125, -2.5938],
         [-2.1562,  2.5625, -1.4766,  ..., -1.0312,  5.5625, -2.4375],
         [-0.7734,  2.3594,  1.0625,  ...,  2.3438,  9.6875, -2.1719]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<MulBackward0>), past_key_values=DynamicCache(), hidden_states=None, attentions=None)


 lm_head_output: 
 tensor([[[-4.6562, -4.4375, -0.0747,  ..., -3.4688, -2.5156, -3.0625],
         [-7.1562, -7.0625,  0.4766,  ..., -6.0312, -4.7812, -4.1250],
         [-6.8438, -7.6250,  0.3867,  ..., -6.0312, -6.6250, -6.0312],
         [-6.4375, -6.7812, -0.2773,  ..., -5.0938, -6.0938, -4.8750],
         [-7.8438, -8.4375,  3.7031,  ..., -9.4375, -8.5000, -6.5000]

In [111]:
token_id = model.generate(input_ids)
tokenizer.batch_decode(token_id)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> The capital of Japan is Tokyo. It is a bustling metropolis with a population of over 13 million people']

In [112]:
token_id = lm_head_output[0,-1].argmax(-1)
tokenizer.decode(token_id)

'Tokyo'

In [113]:
model_output[0].shape

torch.Size([1, 6, 4096])

In [116]:
lm_head_output.shape

torch.Size([1, 6, 32000])

In [120]:
prompt = "Write a mail to send my resume to a HR who is recruiting for Data Scientist role. Also, add my github project details."

#Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors='pt').input_ids
input_ids = input_ids.to("cuda")

In [122]:
%%timeit -n 1
#Generate the text
generation_output = model.generate(
    input_ids = input_ids,
    use_cache=True
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

2.06 s ± 268 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [123]:
%%timeit -n 1
#Generate the text
generation_output = model.generate(
    input_ids = input_ids,
    use_cache=False
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

8.56 s ± 176 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
